In [ ]:
# we load the things!

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
from ngsolve.fem import LeviCivitaSymbol, Einsum

import scipy.sparse as sp
from scipy.optimize import curve_fit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

In [ ]:
# Functions for plotting

def logspace_custom_decades(start, stop, points_per_decade):
    
    result = []
    current_decade = start
    while current_decade < stop:
        decade_points = np.logspace(np.log10(current_decade), np.log10(current_decade * 10), points_per_decade, endpoint=False)
        result.extend(decade_points)
        current_decade *= 10
    return np.array(result)

def reference_line_func(h_values, scaling_factor, slope):
    """
    Reference line model: scaling_factor * h_values ** slope.
    """
    return scaling_factor * h_values ** slope

def fit_reference_line(h_values, error_values):
    """
    Fit a reference line to the error values by finding the optimal scaling factor and slope
    using least-squares regression.
    """
    popt, _ = curve_fit(reference_line_func, h_values, error_values, p0=[1, 1])

    scaling_factor, slope = popt
    return scaling_factor, slope

In [ ]:
# Functions to calculate h_max

def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

In [ ]:
# Hodge Laplace for 1-forms function

def hodgeLaplace1Forms(mesh, 
                       omega_m,
                       g, 
                       order = 1, 
                       C_w = 1 ):
    
    h_curl = HCurl(mesh, order=order, type1=False)  # For 1-forms, H(curl)
    h_1 = H1(mesh, order=order+1)     # For 0-forms, H1 space
    fes = h_curl * h_1
    (omega, sigma), (eta, tau) = fes.TnT()

    a = BilinearForm(fes)

    a += sigma * tau * dx
    a += - omega * grad(tau) * dx

    a +=  grad(sigma) * eta * dx
    a +=  curl(omega) * curl(eta) * dx

    n = specialcf.normal(mesh.dim)
    
    h = specialcf.mesh_size #computed on every edge of the boundary integration is way faster than setting a constant!

    if mesh.dim == 3:
        a += curl(omega) * Cross(eta, n) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
        a += Cross(omega, n) * curl(eta) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
        a += (C_w / h) * Cross(omega, n) * Cross(eta, n) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    
    else:
        tr_hs_d_omega = curl(omega)
        tr_eta = CF(eta[1]*n[0] - eta[0]*n[1])
        tr_omega = CF(omega[1]*n[0] - omega[0]*n[1])
        tr_hs_d_eta = curl(eta)

        a += tr_hs_d_omega * tr_eta * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
        a += tr_omega * tr_hs_d_eta * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
        a += (C_w / h) * tr_omega * tr_eta * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    
    # Hodge Laplcae of Omega Manufactured
    hL_omega_m = CF(GCurl(GCurl(omega_m)) - GGrad(GDiv(omega_m))) 

    f_rhs = LinearForm(fes)
    f_rhs += hL_omega_m * eta * dx

    a.Assemble()
    f_rhs.Assemble()

    rows,cols,vals = a.mat.COO()
    A = sp.csr_matrix((vals,(rows,cols)))
    #cond_nr = np.linalg.cond(A.todense()) # takes way longer than solving the system lolll..

    sol = GridFunction(fes)
    res = f_rhs.vec-a.mat * sol.vec
    inv = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    sol.vec.data += inv * res

    gf_omega , gf_sigma = sol.components

    curl_omega = curl(gf_omega)
    grad_sigma = grad(gf_sigma)

    curl_omega_m = CF(GCurl(omega_m))
    sigma_m = - CF(GDiv(omega_m))
    grad_sigma_m = CF(GGrad(sigma_m))

    L2_error_omega = sqrt(Integrate((gf_omega - omega_m)**2, mesh))
    L2_error_curl_omega = sqrt(Integrate((curl_omega - curl_omega_m)**2, mesh))
    L2_error_sigma = sqrt(Integrate((gf_sigma - sigma_m)**2, mesh))
    L2_error_grad_sigma = sqrt(Integrate((grad_sigma - grad_sigma_m)**2, mesh))

    return fes.ndof, Norm(res), L2_error_omega, L2_error_curl_omega, L2_error_sigma, L2_error_grad_sigma, sol